# "Clasificador de CIFAR-10 usando kNN"
> (SPANISH) Clasificación de imágenes con k Nearest Neighbors y la clase experimental tf.numpy


- toc: true 
- badges: true 
- comments: true
- categories: ["Computer Vision"]
- image: images/stanford.png



## 1. Introducción

Este ejemplo muestra cómo hacer una clasificación de imágenes usando Machine Learning clásico, sin usar una red convolucional. 
Se va a usar k Nearest Neighbors y una precisión básica basada en diferencia de pixels. 

## 2. Setup
Importamos las librerías que vamos a usar. usaremos la función experimental `tensorflow.numpy` para aprovechar las GPUs durante operaciones con funciones `numpy` (por ejemplo, durante la inferencia):

In [ ]:
!pip3 install --quiet --upgrade tf-nightly


In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.experimental.numpy as np
import numpy

from tensorflow.keras import datasets, layers, models

Comprobamos si tenemos GPUs. En caso contrario, no notaremos diferencia de velocidad:

In [ ]:
print("All logical devices:", tf.config.list_logical_devices())
print("All physical devices:", tf.config.list_physical_devices())

## 3. Carga de datos
Cargamos el dataset desde `tensorflow.keras.datasets`:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalizamos valores de píxeles entre 0 y 1
train_images, test_images = train_images / 255.0, test_images / 255.


Visualizamos los primeros 25 elementos del dataset:

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## 4. Preparar datos
El set de entrenamiento es de tamaño `train_images.shape=(50000,32,32,3)` y sus etiquetas `train_labels.shape=(50000, 1)`. Aplanamos el set de entrenamiento y también el de pruebas:

In [ ]:
train_images_rows = train_images.reshape(train_images.shape[0], 32 * 32 * 3) # train_images_rows.shape 50000 x 3072
test_images_rows = test_images.reshape(test_images.shape[0], 32 * 32 * 3) # test_images_rows.shape 10000 x 3072

## 5. Construcción del modelo

In [ ]:
class NearestNeighbor(object):
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = numpy.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
    for i in range(num_test):
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances) # get the index with smallest distance
      print("Training {} of {}".format(i, num_test))
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example

    return Ypred



---



## 6. Entrenamiento
Lanzamos el entrenamiento  usando k Nearest Neighbors (kNN):

In [ ]:
nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(train_images_rows, train_labels) # train the classifier on the training images and labels
Yte_predict = nn.predict(test_images_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print('accuracy: %f' % ( np.mean(Yte_predict == test_labels) ))

## 7. Evaluación

La evaluación se puede hacer con la distancia L1 ó L2. En este caso, usamos la diferencia cuadrática:

In [ ]:
distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))
